# Topic Modeling

This notebook explores the use of **topic modeling** on a dataset of movie summaries. The goal is to identify and group recurring themes across the summaries, providing a deeper understanding of the dataset's structure.
  
To achieve this, we will use **BERTopic**, a powerful topic modeling tool that leverages text embeddings, dimensionality reduction, and density-based clustering to extract interpretable topics from text data.


In [1]:
# Modules to import
import sys
import pandas as pd
import numpy as np

# Paths to add
paths = ['../data','../scripts','../utils']
for path in paths:
    sys.path.append(path)
    
# Data loader
from dataLoader import loadDataframe

# Load data
path_to_directory = '../../data/cleanData/'
df_movies = loadDataframe('movies', path_to_directory)
df_summaries = loadDataframe('summaries', path_to_directory)

c:\Users\arnau\OneDrive\Documents\Scolarité EPFL\ADA\ada-2024-project-teamcsx24\src\models\../data\dataLoader.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_convert] = df[columns_to_convert].applymap(eval)


## Bertopic

### 1) Preprocessing

First, we need to preprocess the text data. Indeed, summaries contain lot of noise and irrelevant terms such as "the", "a", etc. We need to clean the text data before feeding it to the model.
 
1. **Lowercasing**: Converting text to lowercase to ensure uniformity.  
2. **Tokenization**: Breaking down sentences into individual tokens or words.  
3. **Stopword Removal**: Eliminating common words like "the" and "and" that do not contribute to meaning.  
4. **Punctuation Removal**: Stripping away symbols and punctuation marks.  
5. **Lemmatization**: Reducing words to their root forms (e.g., "running" → "run").  
6. **Custom Filtering**: Removing domain-specific terms like "film" and "movie," which are too general to provide meaningful insights.  

We use **spaCy** for tokenization and lemmatization, while **NLTK** provides the list of stopwords for removal. This step ensures the data is clean, compact, and ready for analysis.


In [2]:
import spacy
from nltk.corpus import stopwords
import string
from tqdm import tqdm

In [ ]:
# Load the spaCy model for English language
nlp = spacy.load("en_core_web_sm")

# Download stopwords and punkt tokenizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Load the English stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arnau\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arnau\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# We define our own preprocessing function
def preprocess_text(text):
    # Tokenisation
    doc = nlp(text.lower())  # Convert to lowercase
    
    # Lemmatisation and remove stopwords and punctuation
    processed_tokens = [
        token.lemma_ for token in doc if token.text not in stop_words and token.text not in string.punctuation
    ]
    
    char_to_remove = ["'s", " "]
    processed_tokens = [token for token in processed_tokens if token not in char_to_remove]
    
    word_to_remove = ['film','movie','story','tell','leave']
    processed_tokens = [token for token in processed_tokens if token not in word_to_remove]
    
    return " ".join(processed_tokens)

We apply the preprocessing to each summary of movie.

In [5]:
# Apply the preprocessing to the summaries
tqdm.pandas()
df_cleaned_summaries = df_summaries.copy()
df_cleaned_summaries["cleaned_summary"] = df_cleaned_summaries["summary"].progress_apply(preprocess_text)

# Drop the original summary column
df_cleaned_summaries.drop(columns=["summary"], inplace=True)

  0%|          | 0/42303 [00:00<?, ?it/s]

100%|██████████| 42303/42303 [1:20:13<00:00,  8.79it/s]  


Since our dataset contains summaries in multiple languages, we use the **langdetect** library to identify the language of each summary.  
- Summaries not in English are either removed or translated (in this notebook, we choose to remove them).  
- This ensures the modeling process focuses solely on English text, leading to more consistent and interpretable results.


In [6]:
# Detect the language of each summary
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

df_cleaned_summaries["language"] = df_cleaned_summaries["cleaned_summary"].progress_apply(detect_language)

100%|██████████| 42303/42303 [19:08<00:00, 36.82it/s]


In [7]:
# Save the cleaned summaries
df_cleaned_summaries.to_csv("../../data/cultureData/topicModelData/cleaned_summaries.csv", index=False)

In [ ]:
# Load the cleaned summaries
# df_cleaned_summaries = pd.read_csv("../../data/cultureData/topicModelData/cleaned_summaries.csv")

In [ ]:
pourcentage_english = df_cleaned_summaries["language"].value_counts(normalize=True)["en"] * 100
print('Pourcentage of summaries in English: {:.2f}%'.format(pourcentage_english))

Pourcentage of summaries in English: 99.46%


In [ ]:
# Keep only the summaries in English
df_cleaned_summaries = df_cleaned_summaries[df_cleaned_summaries["language"] == "en"]

### 2) BERTopic Model Creation

In [ ]:
docs = df_cleaned_summaries["cleaned_summary"].tolist()

In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

c:\Users\arnau\anaconda3\envs\ada\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1. Embedding

To represent the summaries numerically, we generate embeddings using the **SentenceTransformer** library and the **all-MiniLM-L6-v2** model.  
- **Text embeddings**: High-dimensional vectors that capture semantic meaning from text.  
- These embeddings allow the model to group similar summaries based on their meanings, rather than just their words.  

The generated embeddings are saved to disk to streamline the workflow and avoid recalculating them in subsequent runs.


In [13]:
# Embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode the summaries
embeddings = embedding_model.encode(docs, show_progress_bar=True)

Batches: 100%|██████████| 1315/1315 [42:35<00:00,  1.94s/it] 


In [14]:
# Save the embeddings
np.save("embeddings.npy", embeddings)

In [ ]:
# Load the embeddings
# embeddings_model = np.load("embeddings.npy")

2. UMAP

Text embeddings are often high-dimensional (e.g., 384 dimensions for the chosen transformer model). To make the clustering step more efficient, we reduce their dimensionality using **UMAP** (Uniform Manifold Approximation and Projection).  
- UMAP preserves the structure of the data while compressing it into a smaller number of dimensions (e.g., 5).  
- This step not only speeds up clustering but also enhances the interpretability of the results.


In [15]:
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

3. HDBSCAN

We cluster the reduced embeddings using **HDBSCAN**, a density-based clustering algorithm.  
- **Advantages of HDBSCAN**:
  - Automatically detects clusters of various sizes and shapes.  
  - Identifies and labels outliers (noise points) that do not belong to any cluster.  
- This step assigns a cluster ID to each summary, which corresponds to a potential topic.  

Clusters identified in this step form the basis for the topics generated by BERTopic.


In [16]:
hdbscan_model = HDBSCAN(min_cluster_size=50, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

4. Vectorizer

While embeddings capture the meaning of the text, traditional **bag-of-words** approaches are useful for understanding the content of each cluster. We use the **CountVectorizer** from scikit-learn to:
- Generate term frequencies within each cluster.  
- Consider both unigrams (single words) and bigrams (pairs of words) for better topic descriptions.  

This textual representation complements the embeddings by enabling interpretable topic names.


In [17]:
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

### 3) BERTopic Model Training

We integrate the components (embeddings, UMAP, HDBSCAN, and CountVectorizer) into a single **BERTopic** model.  
- The model is trained on the cleaned summaries to identify underlying topics.  
- For each summary, BERTopic assigns:
  1. A **topic ID**.
  2. A probability score indicating the confidence of the assignment.  

The trained model is saved for future use.

In [18]:
# Create the model
topic_model = BERTopic(embedding_model=embedding_model,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       vectorizer_model=vectorizer_model,
                       verbose=True)

# Fit the model
topics, probabilities = topic_model.fit_transform(docs)
df_cleaned_summaries['topic'] = topics

2024-11-26 20:49:24,887 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1315/1315 [16:42:18<00:00, 45.73s/it]       
2024-11-27 13:31:50,447 - BERTopic - Embedding - Completed ✓
2024-11-27 13:31:50,450 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-27 13:33:40,868 - BERTopic - Dimensionality - Completed ✓
2024-11-27 13:33:40,881 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-27 13:34:07,625 - BERTopic - Cluster - Completed ✓
2024-11-27 13:34:07,634 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-27 13:34:37,968 - BERTopic - Representation - Completed ✓


In [19]:
# Save the model
topic_model.save("topic_model_min_cluster_size_50")

2024-11-27 13:34:43,258 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


We try different values

### 4) Review Topics

Once topics are generated, we refine them to improve their quality:
1. **Merging similar topics**: Topics with overlapping themes are combined into a single, more coherent topic.  
2. **Dropping non-informative topics**: Topics with low interpretability or relevance are removed.  
3. **Assigning meaningful labels**: Descriptive names are given to each topic based on their most frequent words.  

This step ensures that the final topics are both informative and easy to understand.


In [4]:
df_cleaned_summaries =  pd.read_csv("../../data/cultureData/topicModelData/cleaned_summaries.csv")

# Keep only the summaries in English
df_cleaned_summaries = df_cleaned_summaries[df_cleaned_summaries["language"] == "en"]

docs = df_cleaned_summaries["cleaned_summary"].tolist()

In [16]:
# Load new model
topic_model = BERTopic.load("topic_model_min_cluster_size_50")
topic_info = topic_model.get_topic_info()

In [32]:
# topics to merge

topics_to_merge = [0,2] # love and family
topic_model.merge_topics(docs, topics_to_merge)

topics_to_merge = [16,18] # nazi germany
topic_model.merge_topics(docs, topics_to_merge)

topics_to_merge = [10, 18] # French culture
topic_model.merge_topics(docs, topics_to_merge)

topics_to_merge = [16,26] # pirates
topic_model.merge_topics(docs, topics_to_merge)

topics_to_merge = [7,10] # spain
topic_model.merge_topics(docs, topics_to_merge)

In [33]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,20926,-1_man_kill_make_try,"[man, kill, make, try, time, life, friend, new...",[backwoodsman name adam pontipee new bride mil...
1,0,7433,0_love_father_family_marry,"[love, father, family, marry, life, son, come,...",[young widow commit suicide compel forego mode...
2,1,4606,1_kill_man_house_make,"[kill, man, house, make, car, try, police, hom...",[open michelle mancini drive night storm reali...
3,2,1384,2_murder_police_detective_crime,"[murder, police, detective, crime, killer, pri...",[psychotic killer garland red lynch use campai...
4,3,1165,3_ho_li_master_man,"[ho, li, master, man, china, wong, kill, chine...","[plot|date""farewell concubine study notes"">{{c..."
5,4,673,4_juan_family_miguel_love,"[juan, family, miguel, love, father, man, luis...",[juan oliver want make good impression new job...
6,5,638,5_earth_scientist_planet_human,"[earth, scientist, planet, human, alien, space...",[distant future war human race alien know gami...
7,6,477,6_confederate_town_union_man,"[confederate, town, union, man, war, sheriff, ...",[set state virginia american civil war james s...
8,7,396,7_jean_paris_pierre_marie,"[jean, paris, pierre, marie, love, julien, ant...","["" feel go life like american tourist many tow..."
9,8,356,8_german_nazi_hitler_war,"[german, nazi, hitler, war, berlin, germany, v...",[world war ii wehrmacht colonel claus von stau...


After refining the topics, we associate each summary with its corresponding topic:
- **Topic ID**: A numeric identifier for the topic.  
- **Topic Name**: A descriptive label that summarizes the topic's main theme.  

The updated dataset is saved, with each summary linked to its topic, allowing for further exploration and analysis.

In [11]:
topic_names = {
    -1: "Unclassified",
    0: "Love & Family",
    1: "Crime",
    2: "Investigation",
    3: "Martial Arts",
    4: "Family Drama",
    5: "Sci-Fi Earth",
    6: "Civil War",
    7: "French Life",
    8: "WWII",
    9: "Space",
    10: "Pirates",
    11: "USSR",
    12: "Soldiers",
    13: "Cartoons",
    14: "Monsters",
    15: "Sports",
    16: "Samurai",
    17: "Tom & Jerry",
    18: "Royalty",
    19: "Christmas",
    20: "Stooges",
    21: "Africa",
    22: "Charlie Brown",
    23: "Middle East",
    24: "College",
    25: "Disney",
    26: "Tokyo Life",
    27: "Fantasy",
    28: "Pink Panther",
    29: "Yakuza",
    30: "Jungle",
    31: "Racing",
    32: "Musketeers",
    33: "Laurel & Hardy",
    34: "Betty Boop",
    35: "Godzilla",
    36: "Roman Empire",
    37: "Politics",
    38: "School Life",
    39: "Boxing"
}

topic_model.set_topic_labels(topic_names)

In [36]:
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,20926,-1_man_kill_make_try,Unclassified,"[man, kill, make, try, time, life, friend, new...",[backwoodsman name adam pontipee new bride mil...
1,0,7433,0_love_father_family_marry,Love & Family,"[love, father, family, marry, life, son, come,...",[young widow commit suicide compel forego mode...
2,1,4606,1_kill_man_house_make,Crime,"[kill, man, house, make, car, try, police, hom...",[open michelle mancini drive night storm reali...
3,2,1384,2_murder_police_detective_crime,Investigation,"[murder, police, detective, crime, killer, pri...",[psychotic killer garland red lynch use campai...
4,3,1165,3_ho_li_master_man,Martial Arts,"[ho, li, master, man, china, wong, kill, chine...","[plot|date""farewell concubine study notes"">{{c..."
5,4,673,4_juan_family_miguel_love,Family Drama,"[juan, family, miguel, love, father, man, luis...",[juan oliver want make good impression new job...
6,5,638,5_earth_scientist_planet_human,Sci-Fi Earth,"[earth, scientist, planet, human, alien, space...",[distant future war human race alien know gami...
7,6,477,6_confederate_town_union_man,Civil War,"[confederate, town, union, man, war, sheriff, ...",[set state virginia american civil war james s...
8,7,396,7_jean_paris_pierre_marie,French Life,"[jean, paris, pierre, marie, love, julien, ant...","["" feel go life like american tourist many tow..."
9,8,356,8_german_nazi_hitler_war,WWII,"[german, nazi, hitler, war, berlin, germany, v...",[world war ii wehrmacht colonel claus von stau...


In [37]:
# Save the merged model
topic_model.save("topic_model_merged")

2024-11-28 17:01:37,616 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


Connect each summary to the topic with the highest probability

In [ ]:
topic_model = BERTopic.load("topic_model_merged")

In [7]:
# Get the topics for each summary
topics, probabilities = topic_model.transform(docs)

# Add the topics to the dataframe
df_cleaned_summaries["topic"] = topics

Batches: 100%|██████████| 1315/1315 [36:55<00:00,  1.68s/it] 


In [12]:
# Add the names of the topics
df_cleaned_summaries["topic_name"] = df_cleaned_summaries["topic"].map(topic_names)

In [13]:
# Save the dataframe
df_cleaned_summaries.to_csv("../../data/cultureData/topicModelData/summaries_with_topics.csv", index=False)